#### Project: Estimated Delivery Date Phase II
#### Developer: Vaibhav Prasad Desai
#### Developed on : 07/09/2018
#### Model: Tensorflow :
####                 a.	Linear (Simple Regression)
####                 b.	DNN
####                 c.	Combined DNN

#### import libraries

In [26]:
import os
import pandas as pd
import seaborn as sns
import numpy as np
import random
import math
import datalab.bigquery as bq
import shutil
import tensorflow as tf
import google.datalab.storage as storage
from io import BytesIO
print tf.__version__

1.8.0


#### Import environment variables

In [37]:
BUCKET = 'eddphase2'
PROJECT = 'customerAnalytics-sandbox'
REGION = 'us-east1'

# Dataset variables
DATASET_FILE = 'data.csv'
FILE_TRAIN = './train.csv'
FILE_VALID = './validation.csv'
FILE_TESTS = './test.csv'

#Splitting Variables
PERCENT_TRAIN = 70
PERCENT_VALID = 20
PERCENT_TESTS = 10

#! gsutil cp gs://eddphase2/EDD_VER1.csv ./EDD_VER1.csv

#### Haversine Distance function

In [28]:
def hav_distance(lat1,lat2 ,lon1,lon2):
    
    radius = 3959 # miles

    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c

    return d

#### Function to split dataset into train, Validate and test

In [25]:
def train_validate_test_split(df, train_percent=.7, validate_percent=.2, seed=1234):
    np.random.seed(seed)
    perm = np.random.permutation(df.index)
    m = len(df.index)
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = df.ix[perm[:train_end]]
    validate = df.ix[perm[train_end:validate_end]]
    test = df.ix[perm[validate_end:]]
    return train, validate, test

In [23]:
# Read CSV File
edd_df=pd.read_csv('./EDD_VER1.csv')  


In [29]:
#Getting Distance and adding it as new column
edd_df['Distance'] = edd_df.apply(lambda row: hav_distance(row['FC_LATITUDE'],row['DEST_LATITUDE'],row['FC_LONGITUDE'], row['DEST_LONGITUDE']), axis=1)

In [36]:
# Split the dataset
train, validate, test = train_validate_test_split(edd_df)

/usr/local/envs/py2env/lib/python2.7/site-packages/ipykernel/__main__.py:7: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [58]:
train.head()

,DAY_OF_WEEK,ORDER_TIME,SKU,SHIPMENT_TS,ORDER_FULFILL_STORE_NBR_ID,ORDER_DELIVERY_DT,CITY_NAME,STATECODE,DESTINATION_ZIP,DEST_LONGITUDE,...,SHIP_HOUR,ORDER_SHIP_GAPH,SHIP_DELIVERY_GAPD,ORDER_DELIVERY_GAPD,ORDER_YEAR,ORDER_MONTH,SHIP_WKD,ORDER_M_DAY,DIST_GRP,Distance
1265343,5,2017-04-27 17:08:00.000000,41353073,2017-04-28 01:04:00.000000,657,2017-04-29 00:00:00.000000,EVANS,GA,30809,-82.136764,...,1,8,1,2,2017,4,6,27,101-200,103.205549
1151966,5,2017-09-14 18:33:00.000000,44339036,2017-09-15 00:44:00.000000,653,2017-09-22 00:00:00.000000,BEND,OR,97702,-121.256474,...,0,6,7,8,2017,9,6,14,OVER 1500 PLUS,2382.038503
2461330,6,2017-04-28 12:34:00.000000,47239838,2017-04-29 09:01:00.000000,657,2017-05-03 00:00:00.000000,SAINT LOUIS,MO,63101,-90.191176,...,9,21,4,5,2017,4,7,28,401-500,470.775246
932686,2,2017-06-12 17:19:00.000000,18322641,2017-06-13 00:15:00.000000,653,2017-06-16 00:00:00.000000,MANSFIELD,MA,2048,-71.218787,...,0,7,3,4,2017,6,3,12,101-200,187.132422
5384538,2,2016-11-21 10:23:00.000000,13015368,2016-11-21 16:57:00.000000,677,2016-11-22 00:00:00.000000,CORSICANA,TX,75110,-96.431160,...,16,6,1,1,2016,11,2,21,51-100,78.977100


In [91]:
# export as CSV
train.to_csv(FILE_TRAIN,header=False, sep=',')
validate.to_csv(FILE_VALID, sep=',')
test.to_csv(FILE_TESTS, sep=',')

In [119]:
# CSV_COLUMNS = ['DAY_OF_WEEK', 'SKU','ORDER_FULFILL_STORE_NBR_ID','DEST_LONGITUDE','DEST_LATITUDE','ORDER_HOUR','SHIP_HOUR',
#                'ORDER_SHIP_GAPH','SHIP_DELIVERY_GAPD','ORDER_DELIVERY_GAPD','SHIP_WKD','ORDER_M_DAY','Distance']
# LABEL_COLUMN = 'ORDER_DELIVERY_GAPD'
# DEFAULTS=[[1],[1234560],[653],[-86.0],[41.0],[4],[4],[2],[3],[5],[3],[20],[100.0],[1],[1234560],[653],[-86.0],[41.0],[4],[4],[2],[3],[5],[3],[20],[100.0]]


CSV_COLUMNS = ['DAY_OF_WEEK', 'ORDER_FULFILL_STORE_NBR_ID','DEST_LONGITUDE','DEST_LATITUDE','ORDER_DELIVERY_GAPD']
LABEL_COLUMN = 'ORDER_DELIVERY_GAPD'
DEFAULTS=[[1],[653],[-86.0],[41.0],[4.0]]

In [117]:
train=train[CSV_COLUMNS]
validate=validate[CSV_COLUMNS]
train.to_csv(FILE_TRAIN, header=False,sep=',')
validate.to_csv(FILE_VALID, header=False,sep=',')

In [105]:
train.head()

,DAY_OF_WEEK,ORDER_FULFILL_STORE_NBR_ID,DEST_LONGITUDE,DEST_LATITUDE,ORDER_DELIVERY_GAPD
1265343,5,657,-82.136764,33.541277,2
1151966,5,653,-121.256474,43.972165,8
2461330,6,657,-90.191176,38.631974,5
932686,2,653,-71.218787,42.023917,4
5384538,2,677,-96.431160,32.060160,1


In [106]:
def read_dataset(filename, mode, batch_size = 512):
  def _input_fn():
    def decode_csv(value_column):
      columns = tf.decode_csv(value_column, record_defaults = DEFAULTS)
      features = dict(zip(CSV_COLUMNS, columns))
      label = features.pop(LABEL_COLUMN)
      return features, label

    # Create list of files that match pattern
    file_list = tf.gfile.Glob(filename)
    print('a')
    # Create dataset from file list
    dataset = tf.data.TextLineDataset(file_list).map(decode_csv)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        num_epochs = None # indefinitely
        print(dataset)
        dataset = dataset.shuffle(buffer_size = 10 * batch_size)
    else:
        num_epochs = 1 # end-of-input after this

    dataset = dataset.repeat(num_epochs).batch(batch_size)
    return dataset.make_one_shot_iterator().get_next()
  return _input_fn

In [107]:
def get_train():
  return read_dataset(FILE_TRAIN, mode = tf.estimator.ModeKeys.TRAIN)

In [108]:
INPUT_COLUMNS = [
tf.feature_column.numeric_column('DAY_OF_WEEK'),
tf.feature_column.numeric_column('ORDER_FULFILL_STORE_NBR_ID'),
tf.feature_column.numeric_column('DEST_LONGITUDE'),
tf.feature_column.numeric_column('DEST_LATITUDE'),
]

In [120]:
tf.logging.set_verbosity(tf.logging.INFO)
OUTDIR = 'trained'
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time
model = tf.estimator.LinearRegressor(
      feature_columns = INPUT_COLUMNS, model_dir = OUTDIR)
model.train(input_fn = get_train(), steps = 5);  # TODO: change the name of input_fn as needed

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd31b1d7e50>, '_evaluation_master': '', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': 'trained', '_global_id_in_cluster': 0, '_save_summary_steps': 100}
a
b
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


InvalidArgumentError: Expect 5 fields but have 6 in record 0
	 [[Node: DecodeCSV = DecodeCSV[OUT_TYPE=[DT_INT32, DT_INT32, DT_FLOAT, DT_FLOAT, DT_FLOAT], field_delim=",", na_value="", use_quote_delim=true](arg0, DecodeCSV/record_defaults_0, DecodeCSV/record_defaults_1, DecodeCSV/record_defaults_2, DecodeCSV/record_defaults_3, DecodeCSV/record_defaults_4)]]
	 [[Node: IteratorGetNext = IteratorGetNext[output_shapes=[[?], [?], [?], [?], [?]], output_types=[DT_INT32, DT_FLOAT, DT_FLOAT, DT_INT32, DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](OneShotIterator)]]

In [118]:
def read_dataset(filename, mode, batch_size = 512):
  def _input_fn():
    def decode_csv(value_column):
      columns = tf.decode_csv(value_column, record_defaults = DEFAULTS)
      features = dict(zip(CSV_COLUMNS, columns))
      label = features.pop(LABEL_COLUMN)
      #print(features)
      return features, label
    
    # Create list of files that match pattern
    file_list = tf.gfile.Glob(filename)
    print('a')
    # Create dataset from file list
    dataset = tf.data.TextLineDataset(file_list).map(decode_csv)
    print('b')
    if mode == tf.estimator.ModeKeys.TRAIN:
      num_epochs = None # indefinitely
      dataset = dataset.shuffle(buffer_size = 10 * batch_size)
    else:
      num_epochs = 1 # end-of-input after this
    dataset = dataset.repeat(num_epochs).batch(batch_size)
    return dataset.make_one_shot_iterator().get_next()
  return _input_fn

#### Define Input features

In [75]:
INPUT_COLUMNS = [
tf.feature_column.numeric_column('DAY_OF_WEEK'),
tf.feature_column.numeric_column('DEST_LONGITUDE'),
tf.feature_column.numeric_column('DEST_LATITUDE'),
# tf.feature_column.numeric_column('ORDER_HOUR'),
# tf.feature_column.numeric_column('SHIP_HOUR'),
# tf.feature_column.numeric_column('ORDER_FULFILL_STORE_NBR_ID'),
# tf.feature_column.numeric_column('ORDER_M_DAY'),
]
feature_cols = INPUT_COLUMNS

In [77]:
def make_input_fn(df, num_epochs):
  return tf.estimator.inputs.pandas_input_fn(
  x = df,
  y = df[LABEL_COLUMN],
  batch_size = 128,
  num_epochs = num_epochs,
  shuffle = True,
  queue_capacity = 1000,
  num_threads = 1
  )

In [78]:
tf.logging.set_verbosity(tf.logging.INFO)
OUTDIR = 'trained'
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time
model = tf.estimator.LinearRegressor(
feature_columns = INPUT_COLUMNS, model_dir = OUTDIR)
model.train(input_fn = make_input_fn(train, num_epochs = 10))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd318da06d0>, '_evaluation_master': '', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': 'trained', '_global_id_in_cluster': 0, '_save_summary_steps': 100}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into trained/model.ckpt.
INFO:tensorflow:loss = 3310.0, step = 1
INFO:t

KeyboardInterrupt: 

In [72]:
INPUT_COLUMNS = [
tf.feature_column.numeric_column('DAY_OF_WEEK'),
tf.feature_column.numeric_column('ORDER_FULFILL_STORE_NBR_ID'),
tf.feature_column.numeric_column('DEST_LONGITUDE'),
tf.feature_column.numeric_column('DEST_LATITUDE'),
]

def serving_input_fn():
  feature_placeholders = {
  'DAY_OF_WEEK' : tf.placeholder(tf.int32, [None]),
  'ORDER_FULFILL_STORE_NBR_ID':
  'DEST_LONGITUDE' : tf.placeholder(tf.float32, [None]),
  'DEST_LATITUDE' : tf.placeholder(tf.float32, [None]),
#   'ORDER_HOUR' : tf.placeholder(tf.int32, [None]),
#   'SHIP_HOUR' : tf.placeholder(tf.int32, [None]),
#   'ORDER_FULFILL_STORE_NBR_ID' : tf.placeholder(tf.int32, [None]),
#   'ORDER_M_DAY' : tf.placeholder(tf.int32, [None]),
  }
  features = {
  key: tf.expand_dims(tensor, -1)
  for key, tensor in feature_placeholders.items()
  }
  return tf.estimator.export.ServingInputReceiver(features, feature_placeholders)

In [47]:
def train_and_evaluate(output_dir, num_train_steps):
  estimator = tf.estimator.LinearRegressor(
                            model_dir = output_dir,
                            feature_columns = feature_cols)
  train_spec=tf.estimator.TrainSpec(
                            input_fn = read_dataset(FILE_TRAIN,  mode = tf.estimator.ModeKeys.TRAIN),
                                                    max_steps = num_train_steps)
  exporter = tf.estimator.LatestExporter('exporter', serving_input_fn)
  eval_spec=tf.estimator.EvalSpec(
                            input_fn = read_dataset(FILE_VALID, mode = tf.estimator.ModeKeys.EVAL), 
                            steps = None,
                            start_delay_secs = 1, # start evaluating after N seconds
                            throttle_secs = 10, # evaluate every N seconds
                            exporters = exporter)
  tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

In [73]:
OUTDIR = 'trained'
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time
train_and_evaluate(OUTDIR, num_train_steps = 10)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd31770cf10>, '_evaluation_master': '', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': 'trained', '_global_id_in_cluster': 0, '_save_summary_steps': 100}
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after 10 secs (eval_spec.throttle_secs) or training is finished.
{'SKU': <tf.Tensor 'DecodeCSV:1' shape=() dtype=int32>, 'Distance': <tf.Tensor 'DecodeCSV:12' shape=() dtype=float32>, 'SHIP_DELIVERY_GAP

InvalidArgumentError: Field 1 in record 0 is not a valid int32: DAY_OF_WEEK
	 [[Node: DecodeCSV = DecodeCSV[OUT_TYPE=[DT_INT32, DT_INT32, DT_INT32, DT_FLOAT, DT_FLOAT, ..., DT_INT32, DT_INT32, DT_INT32, DT_INT32, DT_FLOAT], field_delim=",", na_value="", use_quote_delim=true](arg0, DecodeCSV/record_defaults_0, DecodeCSV/record_defaults_1, DecodeCSV/record_defaults_2, DecodeCSV/record_defaults_3, DecodeCSV/record_defaults_4, DecodeCSV/record_defaults_5, DecodeCSV/record_defaults_6, DecodeCSV/record_defaults_7, DecodeCSV/record_defaults_8, DecodeCSV/record_defaults_9, DecodeCSV/record_defaults_10, DecodeCSV/record_defaults_11, DecodeCSV/record_defaults_12, DecodeCSV/record_defaults_13, DecodeCSV/record_defaults_14, DecodeCSV/record_defaults_15, DecodeCSV/record_defaults_16, DecodeCSV/record_defaults_17, DecodeCSV/record_defaults_18, DecodeCSV/record_defaults_19, DecodeCSV/record_defaults_20, DecodeCSV/record_defaults_21, DecodeCSV/record_defaults_22, DecodeCSV/record_defaults_23, DecodeCS